In [2]:
!cd .. && make crowd-unpack

micromamba run -n "bd-dev" bin/crowd.sh -r build/dev -o build/crowd-dev-unpacked -g "BD" unpack
Output directory: build/crowd-dev-unpacked
Output directory: /home/struktured/projects/bravely-default-mod/build/crowd-dev-unpacked
Loaded content.0000.cxi.0000.00000000
Loaded content.0000.cxi.0001.00000001
Loaded cxi/exefs.bin
Loaded cxi/header.bin
Loaded cxi/plain.bin
Loaded cxi/romfs.bin
Loaded cxi/exheader.bin
Loaded cxi/romfs_dir/Common_fr/DReportTable/crowd.fs
Dumping spreadsheet cxi/romfs_dir/Common_fr/DReportTable/crowd.fs
   building cxi/romfs_dir/Common_fr/DReportTable/DarkKnightNoteData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DarkKnightNoteIndex.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableBonusData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableItemData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/DTableMonsterData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/GenomData.btb
   building cxi/romfs_dir/Common_fr/DReportTable/Gen

In [1]:
import sys
import pandas as pd
sys.path.insert(0, "../")

import arcanist
from arcanist.crowd import loader
CROWD_DATA="../build/crowd-dev-unpacked"

data = loader.annotate(crowd_data=CROWD_DATA)


In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

files : list[str] = []
schemas :list[str]= []
for k,v in data.items():    
    for schema in v.keys():
        files.append(k)
        schemas.append(schema)

files_schemas = pd.DataFrame({'files':  files, 'schemas': schemas})

pd.DataFrame({'unique': files_schemas[files_schemas['files'].str.contains('Common_en')].files.unique()})

,unique
0,Common_en/DReportTable
1,Common_en/PartyChat
2,Common_en/Paramater
3,Common_en/Shop
4,Common_en/MiniMapIconTable
5,Common_en/TreasureTable
6,Common_en/MenuTable
7,Common_en/Subtitles
8,Common_en/TextTable
9,Common_en/Battle


In [3]:
battle_data = data['Common/Battle']
battle_data['BuffLimit.btb']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,150,150,150,150,150,150,1000,150,700,700,150,200,200
1,75,75,75,75,75,75,75,75,10,10,75,75,75


In [4]:

from arcanist.crowd.loader import CrowdSchema, CrowdSchemaOverrides

overrides = CrowdSchemaOverrides.builder("Common_en/Battle").add("BuffLimit.btb", index={0: 'MaxPercent', 1:'MinPercent'}, columns={0:'AttackPercent'}).build()

print(overrides)
annot=loader.annotate(crowd_data=data, overrides=overrides)

annot['Common_en/Battle']['BuffLimit.btb'].head()

CrowdSchemaOverrides(path='Common_en/Battle', schemas=[CrowdSchema(sheet='BuffLimit.btb', index={0: 'MaxPercent', 1: 'MinPercent'}, columns={0: 'AttackPercent'})])
Renamed column Unnamed: 0 to AttackPercent in sheet BuffLimit.btb.
Renamed index 0 to MaxPercent in dataset BuffLimit.btb.
Renamed index 1 to MinPercent in dataset BuffLimit.btb.


,AttackPercent,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
MaxPercent,150,150,150,150,150,150,1000,150,700,700,150,200,200
MinPercent,75,75,75,75,75,75,75,75,10,10,75,75,75


In [6]:
loader.CrowdSchemaOverrides.save([overrides])
!cat crowd_overrides.yaml

- !!python/object:arcanist.crowd.loader.CrowdSchemaOverrides
  path: Common_en/Battle
  schemas:
  - !!python/object:arcanist.crowd.loader.CrowdSchema
    columns:
      0: AttackPercent
    index:
      0: MaxPercent
      1: MinPercent
    sheet: BuffLimit.btb


In [ ]:
loader.save(annot)

Saved crowd data to Common_en/DReportTable/crowd.xls
Saved crowd data to Common_en/PartyChat/crowd.xls
Saved crowd data to Common_en/Paramater/crowd.xls
Saved crowd data to Common_en/Shop/crowd.xls
Saved crowd data to Common_en/MiniMapIconTable/crowd.xls
Saved crowd data to Common_en/TreasureTable/crowd.xls
Saved crowd data to Common_en/MenuTable/crowd.xls
Saved crowd data to Common_en/Subtitles/crowd.xls
Saved crowd data to Common_en/TextTable/crowd.xls
Saved crowd data to Common_en/Battle/crowd.xls
Saved crowd data to Common_en/MessageTable/crowd.xls
Saved crowd data to Common_de/DReportTable/crowd.xls
Saved crowd data to Common_de/PartyChat/crowd.xls
Saved crowd data to Common_de/Paramater/crowd.xls
Saved crowd data to Common_de/Shop/crowd.xls
Saved crowd data to Common_de/MiniMapIconTable/crowd.xls
Saved crowd data to Common_de/TreasureTable/crowd.xls
Saved crowd data to Common_de/MenuTable/crowd.xls
Saved crowd data to Common_de/Subtitles/crowd.xls
Saved crowd data to Common_de/Te